In [575]:
import pandas as pd
import numpy as np

In [576]:
candidates_df = pd.read_csv('final-dataset.csv')
seats_df = pd.read_csv('final_seat_availability.csv')

In [577]:
candidates_df

,CET number,Name,Marks,Rank,Category,priority,collegecode
0,115200463,H R SHREE RAKSHA,72,1,GM,1,C487MC
1,115200463,H R SHREE RAKSHA,72,1,GM,2,C574MC
2,115200463,H R SHREE RAKSHA,72,1,GM,3,C446MC
3,115200463,H R SHREE RAKSHA,72,1,GM,4,C536MC
4,115200463,H R SHREE RAKSHA,72,1,GM,5,C576MC
...,...,...,...,...,...,...,...
45245,114200267,KEERTHANA B K,11,15626,GM,6,C479MC
45246,114200267,KEERTHANA B K,11,15626,GM,7,C571MC
45247,114200267,KEERTHANA B K,11,15626,GM,8,C529MC
45248,114200267,KEERTHANA B K,11,15626,GM,9,C491MC


In [578]:
seats_df.columns

Index(['College Code', '1G', '1H', '2AG', '2AH', '2BG', '2BH', '3AG', '3AH',
       '3BG', '3BH', 'GM', 'GMH', 'NKN', 'PH', 'SCG', 'SCH', 'STG', 'STH',
       'XD', 'Total Seats'],
      dtype='object')

In [579]:
cutoff_data = {col: [] for col in seats_df.columns[1:-1]}
cutoff_data

{'1G': [],
 '1H': [],
 '2AG': [],
 '2AH': [],
 '2BG': [],
 '2BH': [],
 '3AG': [],
 '3AH': [],
 '3BG': [],
 '3BH': [],
 'GM': [],
 'GMH': [],
 'NKN': [],
 'PH': [],
 'SCG': [],
 'SCH': [],
 'STG': [],
 'STH': [],
 'XD': []}

In [580]:
iteration_candid = candidates_df[candidates_df['priority']==1]

In [581]:
iteration_candid

,CET number,Name,Marks,Rank,Category,priority,collegecode
0,115200463,H R SHREE RAKSHA,72,1,GM,1,C487MC
10,103200234,ADITI NARASIMHAN,71,2,GM,1,C579MC
20,113200118,DARSHAN PRASHANT HEGDE,70,3,SCG,1,C430MC
30,149200155,ROHIT KOUJALAGI,68,4,1G,1,C525MC
40,101200497,U S ABHIRAM,68,5,STG,1,C463MC
...,...,...,...,...,...,...,...
45200,163200168,MOHAMMED ARMAN AFZAL,14,15622,2AG,1,C475MC
45210,106200040,KARTHIK K,13,15623,1G,1,C531MC
45220,141200139,SHRAVANKUMAR WALI,13,15624,2AG,1,C445MC
45230,107200215,SUSHMITHA H N,13,15625,2BG,1,C469MC


In [582]:
def seat_allocation(candidate, seats):
    # Initialize a set to track allocated candidates
    allocated_candidates = set()
    
    # Initialize cutoff data to keep track of ranks for each college and category
    cutoff_data = {(college_code, category): [] 
                   for college_code in seats['College Code'] 
                   for category in seats.columns[1:-1]}
    
    # Step 1: Allocate GM seats (40% of available GM seats for each college)
    candidate = candidate.sort_values(by=['Rank', 'priority'], ascending=[True, True]).copy()
    candidate['Allocated College'] = None
    candidate['Allocated Category'] = None  # Column to store the allocated category

    # Allocate GM seats for top rankers with priority for that college
    for index, candi in candidate.iterrows():
        college_code = candi['collegecode']
        priority = candi['priority']
        
        # Get available GM seats for the specific college
        avail_gm_seats = seats.loc[seats['College Code'] == college_code, 'GM'].values[0] if college_code in seats['College Code'].values else 0
        
        # Calculate the number of GM seats to allocate (40% of available seats)
        gm_seats_to_allocate = int(avail_gm_seats * 0.4)

        # Allocate GM seats only if the candidate has priority
        if priority <= 10 and gm_seats_to_allocate > 0 and candi['CET number'] not in allocated_candidates:
            candidate.at[index, 'Allocated College'] = college_code
            candidate.at[index, 'Allocated Category'] = 'GM'  # Mark the category as GM
            allocated_candidates.add(candi['CET number'])
            seats.loc[seats['College Code'] == college_code, 'GM'] -= 1  
            gm_seats_to_allocate -= 1
            cutoff_data[(college_code, 'GM')].append(candi['Rank'])

    # Step 2: Proceed with normal allocation
    for index, candi in candidate.iterrows():
        if candi['CET number'] in allocated_candidates:
            continue  # Skip already allocated candidates
        
        category = candi['Category']
        college_code = candi['collegecode']
        avail_seats = seats.loc[seats['College Code'] == college_code, category].values[0] if college_code in seats['College Code'].values else 0

        if avail_seats > 0:
            candidate.at[index, 'Allocated College'] = college_code
            candidate.at[index, 'Allocated Category'] = category  # Mark the allocated category
            allocated_candidates.add(candi['CET number'])
            seats.loc[seats['College Code'] == college_code, category] -= 1  
            cutoff_data[(college_code, category)].append(candi['Rank'])

    # Step 3: Check remaining seats and allocate 25% of them based on priority
    remaining_seats = seats.copy()
    for college_code in remaining_seats['College Code']:
        for category in remaining_seats.columns[1:-1]:  # Exclude 'College Code'
            avail_seats = remaining_seats.loc[remaining_seats['College Code'] == college_code, category].values[0]
            if avail_seats > 0:
                seats_to_allocate = int(avail_seats * 0.25)  # Calculate 25% of remaining seats
                
                # Filter candidates with priority for the current college
                priority_candidates = candidate[(candidate['collegecode'] == college_code) & 
                                                (candidate['priority'] <= 10) & 
                                                (~candidate['CET number'].isin(allocated_candidates))]

                for idx in priority_candidates.index:
                    if seats_to_allocate <= 0:
                        break
                    candidate.at[idx, 'Allocated College'] = college_code
                    candidate.at[idx, 'Allocated Category'] = category  # Allocate based on remaining category
                    allocated_candidates.add(candidate.at[idx, 'CET number'])
                    remaining_seats.loc[remaining_seats['College Code'] == college_code, category] -= 1  
                    seats_to_allocate -= 1
                    cutoff_data[(college_code, category)].append(candidate.at[idx, 'Rank'])

    # Prepare cutoff data after all allocations
    cutoff_rows = []
    for college_c in seats['College Code']:
        row = {'College Code': college_c}
        for category in seats.columns[1:-1]:
            row[category] = max(cutoff_data[(college_c, category)], default="--") if (college_c, category) in cutoff_data else "--"
        cutoff_rows.append(row)
    
    cutoff_df = pd.DataFrame(cutoff_rows)

    # Prepare a candidate list with allocated colleges and categories, including priority and rank
    allocated_candidate_list = candidate[candidate['Allocated College'].notnull()][['CET number', 'Name', 'Allocated College', 'Allocated Category', 'priority', 'Rank']]
    
    # Prepare a candidate list with details of those not allocated any seats
    not_allocated_candidates = candidate[candidate['Allocated College'].isnull()][['CET number', 'Name', 'priority', 'Rank']]

    # Calculate total seats remaining
    remaining_seats['Total Seats'] = remaining_seats.iloc[:, 1:].sum(axis=1)

    return candidate, cutoff_df, remaining_seats, allocated_candidate_list, not_allocated_candidates







allocated_candidates_df, cutoff_df, remaining_seats_df, allocated_candidate_list_df, not_allocated_candidates_df = seat_allocation(candidates_df, seats_df)

remaining_seats_df['Total Seats'] = remaining_seats_df.iloc[:, 1:-1].sum(axis=1)

unique_not_allocated_candidates_df = not_allocated_candidates_df[~not_allocated_candidates_df['CET number'].isin(allocated_candidate_list_df['CET number'])][['CET number', 'Name', 'Rank']]

unique_not_allocated_candidates_df=unique_not_allocated_candidates_df.drop_duplicates()


In [583]:
allocated_candidates_df

,CET number,Name,Marks,Rank,Category,priority,collegecode,Allocated College,Allocated Category
0,115200463,H R SHREE RAKSHA,72,1,GM,1,C487MC,C487MC,GM
1,115200463,H R SHREE RAKSHA,72,1,GM,2,C574MC,None,None
2,115200463,H R SHREE RAKSHA,72,1,GM,3,C446MC,None,None
3,115200463,H R SHREE RAKSHA,72,1,GM,4,C536MC,None,None
4,115200463,H R SHREE RAKSHA,72,1,GM,5,C576MC,None,None
...,...,...,...,...,...,...,...,...,...
45245,114200267,KEERTHANA B K,11,15626,GM,6,C479MC,None,None
45246,114200267,KEERTHANA B K,11,15626,GM,7,C571MC,None,None
45247,114200267,KEERTHANA B K,11,15626,GM,8,C529MC,None,None
45248,114200267,KEERTHANA B K,11,15626,GM,9,C491MC,None,None


In [584]:
cutoff_df

,College Code,1G,1H,2AG,2AH,2BG,2BH,3AG,3AH,3BG,3BH,GM,GMH,NKN,PH,SCG,SCH,STG,STH,XD
0,C412MC,10407,--,15036,--,12976,--,12320,--,8835,--,8264,--,--,10111,9931,14266,15279,--,--
1,C423MC,6200,8604,10280,14675,8097,--,--,10575,--,12537,11468,11589,--,--,13362,10328,5207,--,--
2,C495MC,13147,9732,12507,--,13605,9516,14921,--,12997,--,11020,11779,--,12524,13056,--,15169,12371,13799
3,C496MC,10817,--,12333,10275,11562,--,14856,--,11144,--,9041,11842,--,12382,13409,--,12735,15458,12818
4,C497MC,14137,--,11816,10238,11745,--,15268,--,15194,--,10546,10824,--,12143,13522,--,12277,--,--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,C520MC,5967,--,13131,13380,8267,--,10989,--,15371,--,9306,--,--,--,11413,--,--,--,--
110,C521MC,9601,--,12915,--,6591,--,6527,--,--,--,10107,11500,--,--,8977,--,9784,--,--
111,C543MC,9872,--,15577,--,10883,--,8166,--,--,--,7451,--,--,--,12893,--,14699,--,--
112,C568MC,6896,--,--,--,10999,--,7353,--,7303,--,10483,6258,--,11029,12247,--,--,--,--


In [585]:
remaining_seats_df

,College Code,1G,1H,2AG,2AH,2BG,2BH,3AG,3AH,3BG,...,GM,GMH,NKN,PH,SCG,SCH,STG,STH,XD,Total Seats
0,C412MC,0,0,4,1,0,1,0,0,0,...,0,2,0,2,7,0,0,0,0,17
1,C423MC,0,0,0,3,0,1,0,1,0,...,0,9,0,2,1,4,0,2,0,23
2,C495MC,0,0,15,2,3,0,3,0,4,...,0,5,0,7,20,2,6,0,0,68
3,C496MC,0,0,11,0,0,0,3,0,3,...,0,3,0,5,11,2,3,0,0,42
4,C497MC,0,0,12,0,0,0,2,0,4,...,0,3,0,5,14,2,3,1,0,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,C520MC,0,0,3,0,0,0,0,0,2,...,0,1,0,3,5,1,3,0,0,18
110,C521MC,0,0,0,0,0,1,0,0,1,...,0,0,0,1,0,0,1,0,0,4
111,C543MC,0,0,3,1,0,0,0,0,2,...,0,1,0,3,5,1,0,0,0,16
112,C568MC,0,0,3,1,0,0,0,0,0,...,0,0,0,0,1,0,2,0,0,7


In [586]:
allocated_candidate_list_df

,CET number,Name,Allocated College,Allocated Category,priority,Rank
0,115200463,H R SHREE RAKSHA,C487MC,GM,1,1
10,103200234,ADITI NARASIMHAN,C579MC,GM,1,2
20,113200118,DARSHAN PRASHANT HEGDE,C430MC,GM,1,3
30,149200155,ROHIT KOUJALAGI,C525MC,GM,1,4
40,101200497,U S ABHIRAM,C463MC,GM,1,5
...,...,...,...,...,...,...
45110,115200276,TEJASHWINI B V,C426MC,3BG,1,15593
45164,153200188,MAMTA,C464MC,3AG,5,15601
45181,182200084,SAGAR S,C503MC,SCG,2,15618
45208,163200168,MOHAMMED ARMAN AFZAL,C563MC,2AG,9,15622


In [587]:
unique_not_allocated_candidates_df

,CET number,Name,Rank
18660,183200573,CHIRANTH GOWDA K G,6369
25520,113200058,B CHARAN,8788
25790,160200455,YASHASWINI G J,8866
26870,186200199,LINGAPPA M,9301
29550,102200237,SUDHARSHAN S,10270
...,...,...,...
45170,186200173,HARSHITHA C R,15604
45190,140200252,MALLANA SHREESHAILA CHOUDRY,15621
45210,106200040,KARTHIK K,15623
45230,107200215,SUSHMITHA H N,15625


In [588]:
cutoff_df.to_csv('first-round-cutoff.csv',index=False)